# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import pickle

In [ ]:
# nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


# pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

# classifiers
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# metrics
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import label_ranking_average_precision_score

# visuals
import itertools
import matplotlib.pyplot as plt

nltk.download(['punkt','stopwords','wordnet'])

## Notes: 
+ list table name
+ list column name

In [ ]:
engine = create_engine('sqlite:///DisasterResponse.db')
inspector = inspect(engine)

print(inspector.get_table_names())

for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])

In [ ]:
def load_data(db_path = 'sqlite:///DisasterResponse.db'):
    '''
    Function: Load data from database and return X and y
    Args:
        db_path(str): path of database file name
    Return:
        X: messages for training set
        y: labels of messages for test set
    '''
    # load data from database
    engine = create_engine(db_path)
    df = pd.read_sql_table('labeled_messages' ,engine)
    X = df['message']
    y = df.iloc[:,4:]
    return X, y

In [ ]:
# load data function
X, y = load_data()

In [ ]:
X[:5].values.tolist()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    Function: tokenize the text
    Args:  source string
    Return:
    clean_tokens(str list): clean string list
    
    """
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens  

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
def display_results(y_test, y_pred):
    for i, col in enumerate(y_test.columns.values):
        accuracy = accuracy_score(y_test.loc[:,col], y_pred[:,i])
        print(f'Feature {i+1}: {col} Accuracy: {accuracy:.2f}% \n')
        print(classification_report(y_test[col], y_pred[:,i]))

In [ ]:
# display the test results
display_results(y_test, y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# Show parameters for the pipline
pipeline.get_params()

In [ ]:
# # disk-caching of the output values 
# import joblib
# model = joblib.load("../models/classifier.pkl")
# print(model.best_params_)
# p1 = model.best_estimator_
# p1.get_params()

In [ ]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}
# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=4)

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
pipeline.get_params()

In [ ]:
# parameters = {
#     'vect__smooth_idf': [True,False],
# }
# # create grid search object
# cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
# cv.fit(X_train, y_train)
# y_pred = cv.predict(X_test)   
# display_results(y_test, y_pred)

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = '../models/classifier.pkl'
with open(filename, 'wb') as open_file:
    pickle.dump(cv, open_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import packages
import sys

# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import pickle

# nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


# pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

# classifiers
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# metrics
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import label_ranking_average_precision_score

# visuals
import itertools
import matplotlib.pyplot as plt

nltk.download(['punkt','stopwords','wordnet'])

In [ ]:
def load_data(db_path = 'sqlite:///DisasterResponse.db'):
    '''
    Function: Load data from database and return X and y
    Args:
        db_path(str): path of database file name
    Return:
        X: messages for training set
        y: labels of messages for test set
    '''
    # load data from database
    engine = create_engine(db_path)
    df = pd.read_sql_table('labeled_messages' ,engine)
    X = df['message']
    y = df.iloc[:,4:]
    return X, y


def build_model():
    # text processing and model pipeline
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    #token messages
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    # define parameters for GridSearchCV
    pipeline = Pipeline([
        ('vect',TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,random_state=20)))
    ]) 
    
    parameters = {
        'clf__estimator__criterion':['entropy']
    }

    # create gridsearch object and return as final model pipeline
    model_pipeline = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)

    return model_pipeline


def train(X, y, model):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # fit model
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    # output model test results
    for i, col in enumerate(y_test.columns.values):
        accuracy = accuracy_score(y_test.loc[:,col], y_pred[:,i])
        print(f'Feature {i+1}: {col} Accuracy: {accuracy:.2f}% \n')
        print(classification_report(y_test[col], y_pred[:,i]))

    return model


def export_model(model):
    # Export model as a pickle file
    # save the model to disk
    filename = '../models/classifier.pkl'
    with open(filename, 'wb') as open_file:
        pickle.dump(model, open_file)


def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline